In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import os
# from skimage import io, transform
import pandas as pd
import time
import copy
from torchvision.models import resnet18

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Set up the neural network 

In [7]:
class MeltPoolNetwork(nn.Module):
    def __init__(self, imageModel, num_classes=10, num_param=10):
        super().__init__()
        self.ImageModel = imageModel
        self.paramLayer1 = nn.Sequential(nn.Linear(num_param, 10), nn.Tanh())
        self.paramLayer2 = nn.Sequential(nn.Linear(10, 10), nn.Tanh())
        self.paramLayer3 = nn.Sequential(nn.Linear(10, 10), nn.Tanh())
        self.paramLayer4 = nn.Sequential(nn.Linear(10, 10), nn.Tanh())
        self.fc_withParam = nn.Linear(512 + 10, num_classes)

        tanh_gain = torch.nn.init.calculate_gain('tanh', param=None)
        torch.nn.init.xavier_normal_(self.paramLayer1[0].weight, gain=tanh_gain)
        torch.nn.init.xavier_normal_(self.paramLayer2[0].weight, gain=tanh_gain)
        torch.nn.init.xavier_normal_(self.paramLayer3[0].weight, gain=tanh_gain)
        torch.nn.init.xavier_normal_(self.paramLayer4[0].weight, gain=tanh_gain)
        torch.nn.init.kaiming_normal_(self.fc_withParam.weight, a=0, mode='fan_in', nonlinearity='relu')

    def forward(self, img, pp):
        # Image CNN
        x = self.ImageModel(img)

        # PP NN
        y = self.paramLayer1(pp)
        y = self.paramLayer2(y)
        y = self.paramLayer3(y)
        y = self.paramLayer4(y)
        y = y.view(y.size(0), -1)

        # Combine the process parameters with CNN results
        y = torch.squeeze(y)  # remove any dimensions of 1
        z = torch.cat((x, y), dim=1)
        z = self.fc_withParam(z)
        return z

### Set up the dataset/dataloader 

### Set up the training routine

### Load the data 

### Set up the model and train